# Por que Python?

- **Linguagem mais usada em Data Science**
- Fácil de aprender: sintaxe simples
- Open source
- Grande número de bibliotecas disponíveis
- Desvantagem: desempenho computacional por ser interpretada

# Breve Intro a Python

In [ ]:
# Variaveis sao dinamicamente tipadas
nome = "João"
idade = 45
peso = 85.3

# Listas possuem tipos diferentes e podem ser modificadas
dados = [nome, idade, peso, 5]
print(dados)

# Tuplas possuem tipos diferentes mas não podem ser modificadas
ponto = (1,3,-1)
print(ponto)

# Dicionarios mapeiam uma chave a um valor
dic = {'nome':nome, 'idade':idade, 'peso':peso, 'end': 'Av. Jornalista Anibal Fernandes, Centro de Informática'}
print(dic)
print(f"Nome: {dic['nome']:s}")

In [ ]:
# Testes condicionais
if idade > 65:
    # Indentacao para definir o nivel das instrucoes
    print(nome, "é um candidato a se aposentar")
    print("Erro")
else:
    print(nome, "tem de trabalhar mais um pouco")

In [ ]:

# Repeticoes podem ser feitas de duas formas
# 1 - usando lacos while
vezes = 1
while vezes <= 5:
    print("Já passei por aqui {0} vez(es)".format(vezes))
    vezes += 1

In [ ]:
# 2 - enumerando os elementos de uma lista (objeto iteravel)
for vezes in [1, 2, 3, 4, 5]:
    print("Já passei por aqui {0} vez(es)".format(vezes))
print("====")
# Ou ainda
for vezes in range(1,6,1):
    print("Já passei por aqui {0} vez(es)".format(vezes))

In [ ]:
for dado in dados:
    print(dado)

In [ ]:
for i in range(0,len(dados),1):
    print(dados[i])

In [ ]:
#Slicing
print(dados[2:4])

In [ ]:
# Funcoes em Python
def quadrado(x):
    return x**2

print(quadrado(3))

# Funcoes mais simples podem ser definidas como funcoes lambda
f = lambda x: x**2
print(f(3))

# Processamento Orientado à Coluna

In [ ]:
from sklearn.datasets import california_housing

In [ ]:
data = california_housing.fetch_california_housing()
data.keys()

In [ ]:
print(data["DESCR"])

# Frameworks

## 1- Pandas

Framework para manipulação de dados tabulares em memória

* **Pros**: API extremamente extensiva (inspirada nos dataframes nativos de R)
* **Cons**: single-thread

Operações podem ser feitas "de uma vez só" numa coluna inteira ao invés de ir linha por linha. `pandas` implementa diversas otimizações por debaixo dos panos, de forma que operações como adicionar uma constante para todos valores de uma coluna é feita de forma *quasi*-simultânea

In [ ]:
import pandas as pd

In [ ]:
# Construção de data frames
df = pd.DataFrame(data["data"], columns=data["feature_names"])
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
# Selecionando uma coluna
print(df['Population'].head())

In [ ]:
# Selecionando múltiplas colunas
print(df[['Latitude','Longitude']].head())

In [ ]:
# Selecionando linhas com slice
df[2:5]

In [ ]:
# Seleção de linhas e colunas: loc
print(df.loc[10:15,['Population','AveRooms']])

In [ ]:
# Seleção de linhas e colunas: iloc
print(df.iloc[10:15,[4,2]])

In [ ]:
# Seleção de dados com expressões booleanas
df[(df.Population > 1000) & (df.AveBedrms > 1)].head()

# **Índice**

In [ ]:
df1 = df[(df.Population > 1000) & (df.AveBedrms > 1)]
df1.head()

In [ ]:
print(df1.loc[0:1,['MedInc','HouseAge']])

In [ ]:
df1.reset_index()

In [ ]:
df1.reset_index(inplace=True,drop = True)
df1.head()

In [ ]:
print(df1.loc[0:1,['MedInc','HouseAge']])

In [ ]:
print(df1.iloc[0:2,[0,1]])

<img src="https://raw.githubusercontent.com/ProfLuciano/intro_cd/130b92d33280d3ae7e2e1f34e435b21f8e3a8025/notebooks/pandas_selection.png">

In [ ]:
%%time
# Calculando média de número de quartos usando orientação à linha
accumulator = 0
for record in data["data"]:
    accumulator += record[2]
accumulator /= len(data["data"])
print(f"The mean of mean number of bedrooms in california is {accumulator:.3f}")

In [ ]:
%%time
# Calculando média usando orientada à coluna
mean = df.AveRooms.sum() / len(df)
print(f"The mean of mean number of bedrooms in california is {accumulator:.3f}")

In [ ]:
%%time
# Forma mais simples
mean = df.AveRooms.mean()
print(f"The mean of mean number of bedrooms in california is {mean:.3f}")

In [ ]:
# Função apply: aplica uma função a uma coluna inteira
from math import log
df["HouseAgeLog"] = df.HouseAge.apply(log)
df.head()

Exercícios:

1. Qual o centróide geográfico do dataset? (uma linha!)
2. Quantos quarteirões existem com idade média da casa abaixo de 10 anos e com número de habitantes médio abaixo de 3 pessoas?

In [ ]:
print(df[['Latitude','Longitude']].mean())

In [ ]:
len(df[(df.HouseAge < 10) & (df.AveOccup < 3)])

## 2 - Dask

> Dask is a flexible library for parallel computing in Python.

`dask` é uma alternativa multi-core e/ou distribuída para pandas.
* **Pros** : multi-core, cluster mode, escala
* **Cons**: API limitada, “pensar distribuído”, nem toda operação é trivialmente paralelizada

`dask` segue um modelo *lazy* computacional, então a operação não é de fato executada até que seja chamado `.compute()` explicitamente, retornando um dataframe `pandas` em memória.

In [ ]:
import dask.dataframe as dd
from multiprocessing import cpu_count

N_CORES = cpu_count()
print(f"The machine running this notebook has {N_CORES} cpus.")

ddf = dd.from_pandas(df, npartitions=N_CORES)

In [ ]:
%%time
mean = ddf["AveRooms"].mean().compute()
print(f"The mean of mean number of bedrooms in california is {mean:.3f}")

Quando usar `dask` ao invés de `pandas`?

1. Dataset não couber em memória;
2. `Pandas` estiver lento e se deseja utilizar todos os cores da máquina para paralelizar o processamento;
3. Dataset estiver quebrado em inúmeros arquivos - relacionado a (1)

# Referências

* Docs
    * https://pandas.pydata.org/pandas-docs/stable/
    * http://docs.dask.org/en/latest/


* Designing Data-Intensive Applications
    * https://dataintensive.net 
    * https://www.amazon.com/Designing-Data-Intensive-Applications-Reliable-Maintainable/dp/1449373321


* Extensões do Jupyter notebook: 
    * https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions.html
    * https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html
    * https://towardsdatascience.com/jupyter-notebook-extensions-517fa69d2231
    